# Descargar datos y dependencias

In [ ]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

zsh:1: no matches found: https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
zsh:1: no matches found: https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0
zsh:1: no matches found: https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0
zsh:1: no matches found: https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0
zsh:1: no matches found: https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0


In [1]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
import numpy as np
import pandas as pd
import json
from scipy.sparse import coo_matrix, csr_matrix


/opt/homebrew/lib/python3.10/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


# Cargar Datos

In [2]:
df_books = pd.read_csv('../../Datos/books.csv', sep=',')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [3]:
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

In [4]:
with open("../../Datos/goodreads_past_interactions.json", "r") as f:
    user_interactions = json.load(f)

In [ ]:
with open("../../Datos/goodreads_test_interactions.json", "r") as f:
    user_interactions_test = json.load(f)

In [6]:
idx2userid = {i: id_ for i, id_ in enumerate(user_interactions.keys())}
userid2idx = {id_:i for i, id_ in enumerate(user_interactions.keys())}

In [7]:
rows = []
cols = []
data = []

for user_id, book_ids in user_interactions.items():
    for book_id in book_ids:
        u_id = userid2idx[user_id]
        b_id = bookid2idx[book_id]
        rows.append(int(u_id))
        cols.append(int(b_id))
        data.append(1)

max_user_id = max(rows)
max_item_id = max(cols)

# Crear matriz sparse
user_item_matrix_train = coo_matrix((data, (rows, cols)), shape=(max_user_id + 1, max_item_id + 1))
print(user_item_matrix_train.shape)

rows_test = []
cols_test = []
data_test = []

for user_id, book_ids in user_interactions_test.items():
    for book_id in book_ids:
        u_id = userid2idx[user_id]
        b_id = bookid2idx[book_id]
        rows_test.append(int(u_id))
        cols_test.append(int(b_id))
        data_test.append(1)
    
max_user_id = max([max_user_id] + rows_test)
max_item_id = max([max_item_id] + cols_test)

# Crear matriz sparse
user_item_matrix_test = coo_matrix((data_test, (rows_test, cols_test)), shape=(max_user_id + 1, max_item_id + 1))
print(user_item_matrix_test.shape)

(52821, 4287)
(52821, 4287)


In [ ]:
# 5. Entrenar modelo LightFM
model = LightFM(loss='warp')  # o 'bpr', 'warp-kos', 'logistic'
model.fit(user_item_matrix_train, epochs=20, num_threads=4, verbose=True)

Epoch: 100%|██████████| 20/20 [00:39<00:00,  1.99s/it]


In [10]:
# 6. Evaluar
print("Precision@10 (test):", precision_at_k(model, user_item_matrix_test, k=10).mean())
print("AUC (test):", auc_score(model, user_item_matrix_test).mean())

Precision@10 (test): 0.046
AUC (test): 0.9403147


BERT Embeddings

In [8]:
bert_embeddings = np.load("../../Datos/goodreads_bert_embeddings.npy")  # shape: (num_items, embedding_dim)
#bert_embeddings_large = np.load("goodreads_bert_large_embeddings.npy")
num_items, embedding_dim = bert_embeddings.shape
#num_items, embedding_dim_large = bert_embeddings.shape

print(bert_embeddings.shape)
#print(bert_embeddings_large.shape)

(4287, 768)


In [9]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

bert_embeddings = normalize(bert_embeddings, norm='l2')

In [10]:
book_id_set = set()

for user_id in user_interactions.keys():
  ids = set(user_interactions[user_id])
  book_id_set.update(ids)

print(len(book_id_set))

4287


Usamos PCA porque si no se demora demasiado.

In [11]:
pca_bert_featmat = PCA(n_components=40).fit_transform(bert_embeddings)

item_features_sparse = csr_matrix(bert_embeddings)
item_features_pca_sparse = csr_matrix(pca_bert_featmat)
# concatenate the identity matrix with the PCA features
item_features_sparse = csr_matrix(np.hstack((np.eye(num_items), item_features_sparse.toarray())))
item_features_pca_sparse = csr_matrix(np.hstack((np.eye(num_items), item_features_pca_sparse.toarray())))
print(item_features_sparse.shape)
print(item_features_pca_sparse.shape)
print(num_items+40)

(4287, 5055)
(4287, 4327)
4327


In [12]:
model_bert = LightFM(loss='warp')
model_bert.fit(interactions=user_item_matrix_train, item_features=item_features_pca_sparse, epochs=20, num_threads=4, verbose=True)

Epoch: 100%|██████████| 20/20 [06:24<00:00, 19.24s/it]


Se me demoró como 2.5 minutos con PCA.

In [13]:
# 6. Evaluar
print("Precision@10 (test):", precision_at_k(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse, k=10).mean())
print("AUC (test):", auc_score(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse).mean())

Precision@10 (test): 0.042
AUC (test): 0.9395941


In [16]:
def custom_ranking_metrics_at_k(rec_func, train_user_items, test_user_items, K=10, show_progress=False):

    # Ensure matrices are in CSR format
    train_user_items = train_user_items.tocsr()
    test_user_items = test_user_items.tocsr()

    num_users, num_items = test_user_items.shape
    relevant = 0
    total_precision_div = 0
    total_map = 0
    total_ndcg = 0
    total_auc = 0
    total_users = 0

    # Compute cumulative gain for NDCG normalization
    cg = 1.0 / np.log2(np.arange(2, K + 2))  # Discount factor
    cg_sum = np.cumsum(cg)  # Ideal DCG normalization

    # Get users with at least one item in the test set
    users_with_test_data = np.where(np.diff(test_user_items.indptr) > 0)[0]

    # Progress bar
    #progress = tqdm.tqdm(total=len(users_with_test_data), disable=not show_progress)

    batch_size = 1000
    start_idx = 0

    while start_idx < len(users_with_test_data):
        batch_users = users_with_test_data[start_idx:start_idx + batch_size]
        recommended_items = rec_func(batch_users, N=K)
        start_idx += batch_size

        for user_idx, user_id in enumerate(batch_users):
            test_items = set(test_user_items.indices[test_user_items.indptr[user_id]:test_user_items.indptr[user_id + 1]])

            if not test_items:
                continue  # Skip users without test data

            num_relevant = len(test_items)
            total_precision_div += min(K, num_relevant)

            ap = 0
            hit_count = 0
            auc = 0
            idcg = cg_sum[min(K, num_relevant) - 1]  # Ideal Discounted Cumulative Gain (IDCG)
            num_negative = num_items - num_relevant

            for rank, item in enumerate(recommended_items[user_idx]):
                if item in test_items:
                    relevant += 1
                    hit_count += 1
                    ap += hit_count / (rank + 1)
                    total_ndcg += cg[rank] / idcg
                else:
                    auc += hit_count  # Accumulate hits for AUC calculation

            auc += ((hit_count + num_relevant) / 2.0) * (num_negative - (K - hit_count))
            total_map += ap / min(K, num_relevant)
            total_auc += auc / (num_relevant * num_negative)
            total_users += 1

        #progress.update(len(batch_users))

    #progress.close()

    # Compute final metrics
    precision = relevant / total_precision_div if total_precision_div > 0 else 0
    mean_ap = total_map / total_users if total_users > 0 else 0
    mean_ndcg = total_ndcg / total_users if total_users > 0 else 0
    mean_auc = total_auc / total_users if total_users > 0 else 0

    return {
        "precision": precision,
        "map": mean_ap,
        "ndcg": mean_ndcg,
        "auc": mean_auc
    }

In [32]:
# define a recommendation function for the lightfm model
def rec_function(users, N=10):
    recommendations = []
    for user in users:
        uids = [user]* num_items
        scores = model_bert.predict(uids, np.arange(num_items), item_features=item_features_pca_sparse)
        top_items = np.argsort(-scores)[:N]
        recommendations.append(top_items)
    return np.array(recommendations)

# define a recommendation function for the lightfm model
def rec_function_2(users, N=10):
    recommendations = []
    all_items = np.arange(num_items)
    for user in users:
        past_interactions = user_interactions[idx2userid[user]]
        past_interactions = np.array([bookid2idx[i] for i in past_interactions])
        available_items = np.setdiff1d(all_items, past_interactions)
        uids = [user]* len(available_items)
        scores = model_bert.predict(uids, available_items, item_features=item_features_pca_sparse)
        top_items = np.argsort(-scores)[:N]
        recommendations.append(top_items)
    return np.array(recommendations)

In [33]:
print(custom_ranking_metrics_at_k(rec_function, user_item_matrix_train, user_item_matrix_test, K=10))
print(custom_ranking_metrics_at_k(rec_function_2, user_item_matrix_train, user_item_matrix_test, K=10))

{'precision': 0.042, 'map': 0.014084523809523812, 'ndcg': 0.04361790050279526, 'auc': 0.5198814589665651}
{'precision': 0.03, 'map': 0.007650793650793654, 'ndcg': 0.027361165740072683, 'auc': 0.5138615852232872}
